# OAuth2 Flow Examples

This notebook demonstrates various OAuth2 flows including:
- Authorization Code Flow (with PKCE)
- Client Credentials Flow
- Device Authorization Flow
- Token Exchange Flow

Each flow includes UML sequence diagrams showing the actors and interactions.

Author: Harry Kodden (harry.kodden - at - surf.nl)

## Setup and Dependencies

In [ ]:
# Install required packages
!pip install requests python-dotenv matplotlib plantuml

# For Jupyter notebook display
!pip install IPython

In [ ]:
import os
import json
import time
import base64
import hashlib
import secrets
import requests
from urllib.parse import urlencode, parse_qs, urlparse
from dotenv import load_dotenv
from IPython.display import display, HTML, Image, Markdown
import matplotlib.pyplot as plt

# Load environment variables
load_dotenv('.env')

print("✅ Dependencies loaded successfully!")

## Configuration

In [ ]:
# OAuth2 Server Configuration
OAUTH2_CONFIG = {
    'server_url': os.getenv('OAUTH2_SERVER_URL', 'http://localhost:8080'),
    'host': os.getenv('OAUTH2_SERVER_HOST', 'localhost'),
    'port': int(os.getenv('OAUTH2_SERVER_PORT', 8080))
}

# Client Configurations
CLIENTS = {
    'frontend': {
        'client_id': os.getenv('FRONTEND_CLIENT_ID'),
        'client_secret': os.getenv('FRONTEND_CLIENT_SECRET'),
        'redirect_uri': os.getenv('FRONTEND_REDIRECT_URI'),
        'scopes': os.getenv('DEFAULT_SCOPES', '').split(',')
    },
    'backend': {
        'client_id': os.getenv('BACKEND_CLIENT_ID'),
        'client_secret': os.getenv('BACKEND_CLIENT_SECRET'),
        'scopes': ['api:read', 'api:write']
    },
    'mobile': {
        'client_id': os.getenv('MOBILE_CLIENT_ID'),
        'client_secret': os.getenv('MOBILE_CLIENT_SECRET'),
        'scopes': ['openid', 'profile', 'mobile:access']
    },
    'tv': {
        'client_id': os.getenv('TV_CLIENT_ID'),
        'client_secret': os.getenv('TV_CLIENT_SECRET'),
        'scopes': ['openid', 'profile', 'video:read']
    }
}

# Security Settings
SECURITY = {
    'pkce_enabled': os.getenv('PKCE_ENABLED', 'true').lower() == 'true',
    'state_length': int(os.getenv('STATE_LENGTH', 32)),
    'code_verifier_length': int(os.getenv('CODE_VERIFIER_LENGTH', 43)),
    'http_timeout': int(os.getenv('HTTP_TIMEOUT', 30))
}

print("🔧 Configuration loaded:")
print(f"   Server: {OAUTH2_CONFIG['server_url']}")
print(f"   Clients configured: {list(CLIENTS.keys())}")
print(f"   PKCE enabled: {SECURITY['pkce_enabled']}")

In [ ]:
import iplantuml

## Utility Functions

In [ ]:
def generate_random_string(length: int) -> str:
    """Generate a cryptographically secure random string."""
    return base64.urlsafe_b64encode(secrets.token_bytes(length))[:length].decode('utf-8')

def generate_code_challenge(verifier: str) -> str:
    """Generate PKCE code challenge from verifier."""
    digest = hashlib.sha256(verifier.encode('utf-8')).digest()
    return base64.urlsafe_b64encode(digest).decode('utf-8').rstrip('=')

def display_json(data: dict, title: str = "Response"):
    """Display JSON data in a formatted way."""
    print(f"📋 {title}:")
    print(json.dumps(data, indent=2))
    print()

def display_url(url: str, title: str = "URL"):
    """Display URL as clickable link."""
    display(HTML(f'<p><strong>{title}:</strong></p>'))
    display(HTML(f'<a href="{url}" target="_blank">{url}</a>'))

def check_server_health():
    """Check if OAuth2 server is running."""
    try:
        response = requests.get(f"{OAUTH2_CONFIG['server_url']}/health", timeout=5)
        if response.status_code == 200:
            print("✅ OAuth2 server is running")
            return True
        else:
            print(f"⚠️ OAuth2 server returned status: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Cannot connect to OAuth2 server: {e}")
        return False

print("🛠️ Utility functions loaded")

## Server Health Check

In [ ]:
# Check if the OAuth2 server is running
server_running = check_server_health()

if not server_running:
    display(HTML('<div style="background-color: #ffe6e6; padding: 10px; border-radius: 5px;">'
                 '<strong>⚠️ Warning:</strong> OAuth2 server is not running. '
                 'Please start the server before running the flow examples.'
                 '</div>'))

## 1. Authorization Code Flow with PKCE

The Authorization Code flow is used by web applications and mobile apps to obtain access tokens. PKCE (Proof Key for Code Exchange) adds security for public clients.

### UML Sequence Diagram - Authorization Code Flow

In [ ]:
%%plantuml
@startuml
!theme plain
title Authorization Code Flow with PKCE

actor "User" as User
participant "Client App\n(Frontend)" as Client
participant "User Agent\n(Browser)" as Browser
participant "Authorization Server\n(OAuth2 Server)" as AuthServer
participant "Resource Server\n(API)" as API

== Preparation ==
Client -> Client: Generate code_verifier
Client -> Client: Generate code_challenge = S256(code_verifier)
Client -> Client: Generate state (CSRF protection)

== Authorization Request ==
User -> Client: Initiate login
Client -> Browser: Redirect to /auth with:\n- client_id\n- redirect_uri\n- scope\n- state\n- code_challenge\n- code_challenge_method=S256
Browser -> AuthServer: GET /auth with parameters

== User Authentication ==
AuthServer -> User: Display login form
User -> AuthServer: Provide credentials
AuthServer -> AuthServer: Authenticate user
AuthServer -> User: Display consent screen
User -> AuthServer: Grant consent

== Authorization Response ==
AuthServer -> Browser: Redirect to redirect_uri with:\n- code\n- state
Browser -> Client: GET callback with code & state
Client -> Client: Verify state parameter

== Token Exchange ==
Client -> AuthServer: POST /token with:\n- grant_type=authorization_code\n- code\n- client_id\n- client_secret\n- code_verifier
AuthServer -> AuthServer: Verify code_verifier against code_challenge
AuthServer -> Client: Return tokens:\n- access_token\n- refresh_token\n- id_token

== API Access ==
Client -> API: GET /api/resource\nAuthorization: Bearer access_token
API -> AuthServer: Validate token (optional)
API -> Client: Return protected resource
Client -> User: Display data

@enduml

### Authorization Code Flow Implementation

In [ ]:
class AuthorizationCodeFlowClient:
    def __init__(self, client_config: dict, server_url: str):
        self.client_id = client_config['client_id']
        self.client_secret = client_config['client_secret']
        self.redirect_uri = client_config['redirect_uri']
        self.scopes = client_config['scopes']
        self.server_url = server_url.rstrip('/')
        
    def generate_authorization_url(self, use_pkce: bool = True) -> tuple:
        """Generate authorization URL with PKCE."""
        # Generate state for CSRF protection
        state = generate_random_string(SECURITY['state_length'])
        
        # PKCE parameters
        code_verifier = None
        code_challenge = None
        
        if use_pkce:
            code_verifier = generate_random_string(SECURITY['code_verifier_length'])
            code_challenge = generate_code_challenge(code_verifier)
        
        # Build authorization URL
        params = {
            'response_type': 'code',
            'client_id': self.client_id,
            'redirect_uri': self.redirect_uri,
            'scope': ' '.join(self.scopes),
            'state': state
        }
        
        if use_pkce and code_challenge:
            params['code_challenge'] = code_challenge
            params['code_challenge_method'] = 'S256'
        
        auth_url = f"{self.server_url}/auth?{urlencode(params)}"
        
        return auth_url, state, code_verifier
    
    def exchange_code_for_tokens(self, authorization_code: str, state: str, code_verifier: str = None) -> dict:
        """Exchange authorization code for tokens."""
        data = {
            'grant_type': 'authorization_code',
            'code': authorization_code,
            'redirect_uri': self.redirect_uri,
            'client_id': self.client_id,
            'client_secret': self.client_secret
        }
        
        if code_verifier:
            data['code_verifier'] = code_verifier
        
        response = requests.post(
            f"{self.server_url}/token",
            data=data,
            timeout=SECURITY['http_timeout']
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Token exchange failed: {response.status_code} - {response.text}")

# Create client instance
auth_code_client = AuthorizationCodeFlowClient(CLIENTS['frontend'], OAUTH2_CONFIG['server_url'])

print("🔐 Authorization Code Flow client created")

In [ ]:
# Generate authorization URL
auth_url, state, code_verifier = auth_code_client.generate_authorization_url(use_pkce=True)

print("🚀 Authorization Code Flow - Step 1: Authorization Request")
print(f"📋 Client ID: {auth_code_client.client_id}")
print(f"🔗 Redirect URI: {auth_code_client.redirect_uri}")
print(f"🎯 Scopes: {', '.join(auth_code_client.scopes)}")
print(f"🔒 PKCE enabled: {code_verifier is not None}")
print(f"🎲 State: {state}")
print()

display_url(auth_url, "Authorization URL")

display(HTML('''
<div style="background-color: #e6f3ff; padding: 15px; border-radius: 5px; margin: 10px 0;">
    <h4>📱 Instructions:</h4>
    <ol>
        <li>Click the authorization URL above</li>
        <li>Complete the authentication process</li>
        <li>Copy the authorization code from the callback URL</li>
        <li>Paste it in the next cell</li>
    </ol>
</div>
'''))

In [ ]:
# Manual input for authorization code (in real app, this would be automatic)
authorization_code = input("📝 Enter the authorization code from the callback URL: ")

if authorization_code:
    try:
        print("\n🔄 Exchanging authorization code for tokens...")
        tokens = auth_code_client.exchange_code_for_tokens(authorization_code, state, code_verifier)
        
        print("✅ Token exchange successful!")
        display_json(tokens, "Access Tokens")
        
        # Store tokens for later use
        access_token = tokens.get('access_token')
        refresh_token = tokens.get('refresh_token')
        
    except Exception as e:
        print(f"❌ Token exchange failed: {e}")
else:
    print("⚠️ No authorization code provided")

## 2. Client Credentials Flow

The Client Credentials flow is used for server-to-server authentication where no user interaction is required.

### UML Sequence Diagram - Client Credentials Flow

In [ ]:
%%plantuml
!theme plain
title Client Credentials Flow (Service-to-Service)

participant "Client Application\n(Backend Service)" as Client
participant "Authorization Server\n(OAuth2 Server)" as AuthServer
participant "Resource Server\n(API)" as API

== Token Request ==
Client -> AuthServer: POST /token with:\n- grant_type=client_credentials\n- client_id\n- client_secret\n- scope (optional)
AuthServer -> AuthServer: Validate client credentials
AuthServer -> Client: Return access_token:\n- access_token\n- token_type\n- expires_in\n- scope

== API Access ==
loop Multiple API calls
    Client -> API: GET/POST /api/resource\nAuthorization: Bearer access_token
    API -> AuthServer: Validate token (optional)
    API -> Client: Return protected resource
end

== Token Refresh (when expired) ==
Client -> AuthServer: POST /token with:\n- grant_type=client_credentials\n- client_id\n- client_secret
AuthServer -> Client: Return new access_token

@enduml

### Client Credentials Flow Implementation

In [ ]:
class ClientCredentialsFlowClient:
    def __init__(self, client_config: dict, server_url: str):
        self.client_id = client_config['client_id']
        self.client_secret = client_config['client_secret']
        self.scopes = client_config['scopes']
        self.server_url = server_url.rstrip('/')
        
    def get_access_token(self) -> dict:
        """Get access token using client credentials."""
        data = {
            'grant_type': 'client_credentials',
            'client_id': self.client_id,
            'client_secret': self.client_secret,
            'scope': ' '.join(self.scopes)
        }
        
        response = requests.post(
            f"{self.server_url}/token",
            data=data,
            timeout=SECURITY['http_timeout']
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Token request failed: {response.status_code} - {response.text}")
    
    def test_api_call(self, access_token: str, endpoint: str = "/api/test") -> dict:
        """Test API call with access token."""
        headers = {
            'Authorization': f'Bearer {access_token}',
            'Content-Type': 'application/json'
        }
        
        response = requests.get(
            f"{self.server_url}{endpoint}",
            headers=headers,
            timeout=SECURITY['http_timeout']
        )
        
        return {
            'status_code': response.status_code,
            'headers': dict(response.headers),
            'body': response.text
        }

# Create client instance
client_creds_client = ClientCredentialsFlowClient(CLIENTS['backend'], OAUTH2_CONFIG['server_url'])

print("🔐 Client Credentials Flow client created")

In [ ]:
# Execute Client Credentials Flow
print("🚀 Client Credentials Flow - Service-to-Service Authentication")
print(f"📋 Client ID: {client_creds_client.client_id}")
print(f"🎯 Scopes: {', '.join(client_creds_client.scopes)}")
print()

try:
    print("🔄 Requesting access token...")
    tokens = client_creds_client.get_access_token()
    
    print("✅ Token request successful!")
    display_json(tokens, "Access Token Response")
    
    # Test API call
    access_token = tokens.get('access_token')
    if access_token:
        print("🧪 Testing API access...")
        api_response = client_creds_client.test_api_call(access_token)
        display_json(api_response, "API Response")
        
except Exception as e:
    print(f"❌ Client Credentials Flow failed: {e}")

## 3. Device Authorization Flow

The Device flow is used by input-constrained devices (like smart TVs, IoT devices) that cannot easily display web pages or handle redirects.

### UML Sequence Diagram - Device Flow

In [ ]:
%%plantuml
@startuml
!theme plain
title Device Authorization Flow (RFC 8628)

actor "User" as User
participant "Device\n(Smart TV/IoT)" as Device
participant "User Agent\n(Phone/Computer)" as Browser
participant "Authorization Server\n(OAuth2 Server)" as AuthServer
participant "Resource Server\n(API)" as API

== Device Authorization Request ==
User -> Device: Initiate login
Device -> AuthServer: POST /device_authorization with:\n- client_id\n- scope
AuthServer -> AuthServer: Generate device_code & user_code
AuthServer -> Device: Return:\n- device_code\n- user_code\n- verification_uri\n- expires_in\n- interval

== User Instructions ==
Device -> User: Display:\n- user_code\n- verification_uri\n"Go to verification_uri and enter user_code"

== User Authorization ==
User -> Browser: Navigate to verification_uri
Browser -> AuthServer: GET /device
AuthServer -> User: Display user_code input form
User -> AuthServer: Enter user_code
AuthServer -> AuthServer: Validate user_code
AuthServer -> User: Display login form
User -> AuthServer: Provide credentials
AuthServer -> AuthServer: Authenticate user
AuthServer -> User: Display consent screen
User -> AuthServer: Grant consent
AuthServer -> AuthServer: Mark device_code as authorized

== Device Polling ==
loop Every 'interval' seconds
    Device -> AuthServer: POST /token with:\n- grant_type=urn:ietf:params:oauth:grant-type:device_code\n- device_code\n- client_id
    alt User hasn't authorized yet
        AuthServer -> Device: 400 authorization_pending
    else User authorized
        AuthServer -> Device: 200 with tokens:\n- access_token\n- refresh_token\n- id_token
    end
end

== API Access ==
Device -> API: GET /api/resource\nAuthorization: Bearer access_token
API -> Device: Return protected resource
Device -> User: Display content

@enduml

### Device Flow Implementation

In [ ]:
class DeviceFlowClient:
    def __init__(self, client_config: dict, server_url: str):
        self.client_id = client_config['client_id']
        self.client_secret = client_config['client_secret']
        self.scopes = client_config['scopes']
        self.server_url = server_url.rstrip('/')
        
    def start_device_authorization(self) -> dict:
        """Start device authorization flow."""
        data = {
            'client_id': self.client_id,
            'scope': ' '.join(self.scopes)
        }
        
        response = requests.post(
            f"{self.server_url}/device_authorization",
            data=data,
            timeout=SECURITY['http_timeout']
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Device authorization failed: {response.status_code} - {response.text}")
    
    def poll_for_token(self, device_code: str, interval: int = 5, max_attempts: int = 20) -> dict:
        """Poll for access token."""
        data = {
            'grant_type': 'urn:ietf:params:oauth:grant-type:device_code',
            'device_code': device_code,
            'client_id': self.client_id
        }
        
        for attempt in range(max_attempts):
            response = requests.post(
                f"{self.server_url}/token",
                data=data,
                timeout=SECURITY['http_timeout']
            )
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 400:
                error_data = response.json()
                if error_data.get('error') == 'authorization_pending':
                    print(f"⏳ Attempt {attempt + 1}: Waiting for user authorization...")
                    time.sleep(interval)
                    continue
                else:
                    raise Exception(f"Authorization failed: {error_data}")
            else:
                raise Exception(f"Token request failed: {response.status_code} - {response.text}")
        
        raise Exception("Polling timeout: User did not authorize within the time limit")

# Create client instance
device_client = DeviceFlowClient(CLIENTS['tv'], OAUTH2_CONFIG['server_url'])

print("📺 Device Flow client created")

In [ ]:
# Execute Device Flow
print("🚀 Device Authorization Flow - Smart TV Example")
print(f"📋 Client ID: {device_client.client_id}")
print(f"🎯 Scopes: {', '.join(device_client.scopes)}")
print()

try:
    print("🔄 Starting device authorization...")
    device_auth = device_client.start_device_authorization()
    
    print("✅ Device authorization started!")
    display_json(device_auth, "Device Authorization Response")
    
    # Display user instructions
    user_code = device_auth.get('user_code')
    verification_uri = device_auth.get('verification_uri')
    device_code = device_auth.get('device_code')
    expires_in = device_auth.get('expires_in', 600)
    interval = device_auth.get('interval', 5)
    
    display(HTML(f'''
    <div style="background-color: #fff3cd; padding: 20px; border-radius: 10px; margin: 15px 0; border-left: 5px solid #ffc107;">
        <h3>📺 Device Authorization Instructions</h3>
        <p><strong>👤 User Code:</strong> <code style="font-size: 1.2em; background: #f8f9fa; padding: 5px;">{user_code}</code></p>
        <p><strong>🌐 Verification URL:</strong> <a href="{verification_uri}" target="_blank">{verification_uri}</a></p>
        <p><strong>⏰ Expires in:</strong> {expires_in} seconds</p>
        <hr>
        <h4>📱 Steps:</h4>
        <ol>
            <li>Open the verification URL in your browser</li>
            <li>Enter the user code: <strong>{user_code}</strong></li>
            <li>Complete the authentication and authorization</li>
            <li>Return here to see the polling results</li>
        </ol>
    </div>
    '''))
    
except Exception as e:
    print(f"❌ Device authorization failed: {e}")

In [ ]:
# Start polling for tokens (run this after completing authorization)
if 'device_code' in locals():
    try:
        print("🔄 Polling for access token...")
        print("💡 Complete the authorization in your browser first!")
        
        tokens = device_client.poll_for_token(device_code, interval, max_attempts=10)
        
        print("\n✅ Device flow completed successfully!")
        display_json(tokens, "Access Tokens")
        
    except Exception as e:
        print(f"❌ Device flow failed: {e}")
else:
    print("⚠️ Run the previous cell first to start device authorization")

## 4. Token Exchange Flow

The Token Exchange flow (RFC 8693) allows exchanging one token for another, enabling scenarios like token delegation and impersonation.

### UML Sequence Diagram - Token Exchange Flow

In [ ]:
%%plantuml
@startuml
!theme plain
title Token Exchange Flow (RFC 8693) - Delegation Scenario

actor "User" as User
participant "Frontend Service" as Frontend
participant "Backend Service" as Backend
participant "Authorization Server\n(OAuth2 Server)" as AuthServer
participant "Downstream Service" as Downstream

== Initial Authentication ==
User -> Frontend: Login request
Frontend -> AuthServer: Authorization Code Flow
AuthServer -> Frontend: Return user access_token

== Service-to-Service Call ==
Frontend -> Backend: API call with user token
Backend -> Backend: Need to call downstream service\nwith different audience/scope

== Token Exchange ==
Backend -> AuthServer: POST /token with:\n- grant_type=urn:ietf:params:oauth:grant-type:token-exchange\n- subject_token (user's token)\n- subject_token_type=access_token\n- audience=downstream-service\n- scope=limited-scope\n- client_id & client_secret
AuthServer -> AuthServer: Validate subject_token\nCheck delegation permissions\nGenerate new token for audience
AuthServer -> Backend: Return new access_token\nfor downstream service

== Downstream Call ==
Backend -> Downstream: API call with new token\nAuthorization: Bearer new_access_token
Downstream -> AuthServer: Validate token (optional)
Downstream -> Backend: Return protected resource
Backend -> Frontend: Return aggregated response
Frontend -> User: Display result

note over AuthServer
  Token Exchange can also support:
  - Impersonation (with actor_token)
  - Token type conversion
  - Scope reduction
  - Audience restriction
end note

@enduml

### Token Exchange Flow Implementation

In [ ]:
class TokenExchangeFlowClient:
    def __init__(self, client_config: dict, server_url: str):
        self.client_id = client_config['client_id']
        self.client_secret = client_config['client_secret']
        self.server_url = server_url.rstrip('/')
        
    def exchange_token(self, subject_token: str, audience: str = None, 
                      scopes: list = None, actor_token: str = None) -> dict:
        """Exchange a token for a new token with different properties."""
        data = {
            'grant_type': 'urn:ietf:params:oauth:grant-type:token-exchange',
            'subject_token': subject_token,
            'subject_token_type': 'urn:ietf:params:oauth:token-type:access_token',
            'client_id': self.client_id,
            'client_secret': self.client_secret
        }
        
        if audience:
            data['audience'] = audience
            
        if scopes:
            data['scope'] = ' '.join(scopes)
            
        if actor_token:
            data['actor_token'] = actor_token
            data['actor_token_type'] = 'urn:ietf:params:oauth:token-type:access_token'
        
        response = requests.post(
            f"{self.server_url}/token",
            data=data,
            timeout=SECURITY['http_timeout']
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Token exchange failed: {response.status_code} - {response.text}")

# Create client instance
token_exchange_client = TokenExchangeFlowClient(CLIENTS['backend'], OAUTH2_CONFIG['server_url'])

print("🔄 Token Exchange Flow client created")

In [ ]:
# Demo Token Exchange Flow
print("🚀 Token Exchange Flow - Delegation Example")
print(f"📋 Client ID: {token_exchange_client.client_id}")
print()

# First, get a token to exchange (using client credentials)
try:
    print("1️⃣ Getting initial token (subject token)...")
    initial_tokens = client_creds_client.get_access_token()
    subject_token = initial_tokens.get('access_token')
    
    print(f"✅ Got subject token: {subject_token[:30]}...")
    
    print("\n2️⃣ Exchanging token for downstream service...")
    exchanged_tokens = token_exchange_client.exchange_token(
        subject_token=subject_token,
        audience="downstream-service",
        scopes=["api:read"]  # Reduced scope
    )
    
    print("✅ Token exchange successful!")
    display_json(exchanged_tokens, "Exchanged Tokens")
    
    # Compare tokens
    print("\n📊 Token Comparison:")
    print(f"Original token scope: {initial_tokens.get('scope', 'N/A')}")
    print(f"Exchanged token scope: {exchanged_tokens.get('scope', 'N/A')}")
    print(f"Issued token type: {exchanged_tokens.get('issued_token_type', 'N/A')}")
    
except Exception as e:
    print(f"❌ Token exchange failed: {e}")
    print("💡 Note: Token exchange might not be fully implemented in the demo server")

## 5. Token Introspection and Management

In [ ]:
def introspect_token(token: str, client_id: str, client_secret: str) -> dict:
    """Introspect a token to get its metadata."""
    data = {
        'token': token,
        'client_id': client_id,
        'client_secret': client_secret
    }
    
    response = requests.post(
        f"{OAUTH2_CONFIG['server_url']}/introspect",
        data=data,
        timeout=SECURITY['http_timeout']
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Token introspection failed: {response.status_code} - {response.text}")

def revoke_token(token: str, client_id: str, client_secret: str) -> bool:
    """Revoke a token."""
    data = {
        'token': token,
        'client_id': client_id,
        'client_secret': client_secret
    }
    
    response = requests.post(
        f"{OAUTH2_CONFIG['server_url']}/revoke",
        data=data,
        timeout=SECURITY['http_timeout']
    )
    
    return response.status_code == 200

# Test token introspection if we have a token
if 'access_token' in locals():
    try:
        print("🔍 Token Introspection Example")
        introspection_result = introspect_token(
            access_token, 
            CLIENTS['backend']['client_id'], 
            CLIENTS['backend']['client_secret']
        )
        display_json(introspection_result, "Token Introspection Result")
        
    except Exception as e:
        print(f"❌ Token introspection failed: {e}")
else:
    print("💡 Run a flow example first to get a token for introspection")

## 6. OAuth2 Discovery Document

In [ ]:
def get_discovery_document() -> dict:
    """Get OAuth2 discovery document."""
    response = requests.get(
        f"{OAUTH2_CONFIG['server_url']}/.well-known/oauth-authorization-server",
        timeout=SECURITY['http_timeout']
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Discovery failed: {response.status_code} - {response.text}")

# Get and display discovery document
try:
    print("🔍 OAuth2 Server Discovery Document")
    discovery = get_discovery_document()
    display_json(discovery, "Discovery Document")
    
    # Highlight key endpoints
    print("🔗 Key Endpoints:")
    endpoints = {
        'Authorization': discovery.get('authorization_endpoint'),
        'Token': discovery.get('token_endpoint'),
        'UserInfo': discovery.get('userinfo_endpoint'),
        'Device Authorization': discovery.get('device_authorization_endpoint'),
        'Token Introspection': discovery.get('introspection_endpoint'),
        'Token Revocation': discovery.get('revocation_endpoint')
    }
    
    for name, endpoint in endpoints.items():
        if endpoint:
            print(f"   {name}: {endpoint}")
    
except Exception as e:
    print(f"❌ Discovery failed: {e}")

## Summary

This notebook demonstrated the following OAuth2 flows:

### 🔐 Authorization Code Flow with PKCE
- **Use Case**: Web applications, mobile apps
- **Security**: PKCE prevents code interception attacks
- **User Interaction**: Required (browser-based)

### 🤖 Client Credentials Flow
- **Use Case**: Service-to-service authentication
- **Security**: Client secret protection
- **User Interaction**: None (machine-to-machine)

### 📺 Device Authorization Flow
- **Use Case**: Input-constrained devices (Smart TV, IoT)
- **Security**: Out-of-band user authorization
- **User Interaction**: On secondary device

### 🔄 Token Exchange Flow
- **Use Case**: Token delegation, impersonation
- **Security**: Controlled token exchange with audience/scope restrictions
- **User Interaction**: None (server-side)

### 🛠️ Additional Features
- Token introspection and revocation
- OAuth2 discovery document
- UML sequence diagrams for each flow

Each flow serves different use cases and security requirements in modern OAuth2 implementations.